In [1]:
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
import numpy as np
import EoN

In [2]:
#read edge list from csv file
edges_list = pd.read_csv('../Datasets/lesmis.csv')

#create graph from the edge list
original_G = nx.from_pandas_edgelist(edges_list, edge_attr='weight', create_using=nx.Graph())

#convert the graph into a directed graph
G = original_G.to_directed()

In [3]:
#calculate strength for the nodes
node_strength = {node : sum([G[u][v]['weight'] for u,v in G.edges(node)]) for node in G.nodes()}

#add strength attribute for each node
nx.set_node_attributes(G, node_strength, name="strength")

In [4]:
#calculate the normalized weight for each edge
normalized_weight = {(u,v): G[u][v]['weight']/G.nodes[u]['strength'] for u, v in G.edges()}

#add the normalized weight attribute for each edge
nx.set_edge_attributes(G, normalized_weight, name="w/s")

In [5]:
#extract the degrees of the network
degrees = [original_G.degree(n) for n in original_G.nodes()]
    
#calculte the average degree i.e the first moment
first_moment = np.average(degrees)
    
#calculate the variance of the degrees i.e the second moment
x = 0
for i in range(len(degrees)):
    x = x + degrees[i]**2
second_moment = x / len(degrees)
    
#calculate the epidemic threshold using the first and the second degree
epidemic_thresh = first_moment / (second_moment - first_moment)
    
#set the number of simulations, infection, and recovery rate for the sir model
nb_simulations = 100000
tau = epidemic_thresh
gamma = 0.01

In [6]:
#define the simulation trees dictionary
simulation_trees = {}

#simulate the epidemic nb_simulations times
for i in range(nb_simulations):
    
    #run the simulatin with the defined infection, recovery rate, and the normalized weight as the transmission weight
    sim = EoN.fast_SIR(G, tau, gamma, transmission_weight='w/s', return_full_data=True)
    
    #extract the transmission tree from the simulation result
    simulation_trees[i] = sim.transmission_tree()

    #visualize the transmission tree
    # Tpos = EoN.hierarchy_pos(T)
    # fig = plt.figure(figsize = (8,5))
    # ax = fig.add_subplot(111)
    # nx.draw(T, Tpos, ax=ax, node_size = 200, with_labels=True)
    # plt.show()

In [7]:
#set a new edge attribute for counting the infection participation
nx.set_edge_attributes(G, 0, name="infection_participation")

#count the infection paritcipation for each edge
for sim in simulation_trees:
    tree = simulation_trees[sim]
    for u, v in tree.edges():
        G[u][v]['infection_participation'] = G[u][v]['infection_participation'] + 1

In [8]:
#set a new edge attribute for counting the infection participation
nx.set_edge_attributes(original_G, 0, name="infection_participation")

#add the results to the original graph
for u, v in original_G.edges():
    original_G[u][v]['infection_participation'] = (G[u][v]['infection_participation'] + G[v][u]['infection_participation'])/nb_simulations

In [12]:
#create an edge list from the result graph with the infection participation values
results = nx.to_pandas_edgelist(original_G)

#save the edge list to visualize later
results.to_csv('../Results/Epidemic Simulations Resutls/lesmis_epidemic_simulations.csv', index=False)